### Тестовая задание в НПЦ «БизнесАвтоматика». Герасмиов К.М. 

## 1.1 Задача про регионы
Для выполнения задания: 
1. Создадим датафреймы: user, regions, users_regions 
2. Зальём их на тестовоую базу SQLite
3. Напишем SQL запрос запрос по условию:  
>написать SQL-запрос, получающий список регионов, доступных пользователю в системе. Если пользователь не имеет связи с регионом, должны выводиться все регионы системы. При этом: 1. идентификатор пользователя передается в запрос в виде переменной; 2. в таблице UsersRegions нет дубликатов 


In [1]:
#для формирования датафремов
import pandas as pd
#для выполнения SQL запроса
from sqlalchemy import create_engine

In [2]:
#таблица с пользователям
user = {
    'id': [1, 2, 3],
    'fio': ['Иванов Иван Иванович', 'Иванов Иван Иванович', 'Федоров Федор Федорович']
}
user = pd.DataFrame(user) 
#таблица с регионами
regions = {
    'id': [1, 2, 3],
    'title': ['Москва', 'Липецкая область', 'Республика Саха']
}
regions = pd.DataFrame(regions) 
#таблица связь пользователей с регионами
users_regions = {
    'id': [1, 2, 3],
    'user_id': [1, 2, 2],
    'region_id': [1, 2, 3]
}
users_regions = pd.DataFrame(users_regions) 

display(user)
display(regions)
display(users_regions)

,id,fio
0,1,Иванов Иван Иванович
1,2,Иванов Иван Иванович
2,3,Федоров Федор Федорович


,id,title
0,1,Москва
1,2,Липецкая область
2,3,Республика Саха


,id,user_id,region_id
0,1,1,1
1,2,2,2
2,3,2,3


In [3]:
#создаём движок для БД 
engine = create_engine('sqlite:///mydatabase.db')
#сохраняем наши таблицы в БД
user.to_sql('user', engine, if_exists='replace', index=False)
regions.to_sql('regions', engine, if_exists='replace', index=False)
users_regions.to_sql('users_regions', engine, if_exists='replace', index=False) 

3

Таблицы сформированы, теперь создадим переменную, которая будет принимать значения идентификатора пользователя и выполним SQL запрос, отвечающий на поставленную задачу

In [4]:
input_id = 99

In [5]:
# коннектимся к базе и выполняем sql запрос
# в самом запросе создаём 2 временные таблицы: в t1 проверям вхождения переменной по user_id
# в t2 прописываем условие, если вхождений не было выводим полный список регинов
#если было, то только те по которым есть связь
#с помощью дискинт убираем лишние значения, которые добавились в результате формирования таблицы t2 
with engine.connect() as connection:
    query = f'''
        WITH t1 AS (SELECT ur.user_id,
                           r.title AS title
        FROM users_regions ur
        JOIN user u ON u.id=ur.user_id
        JOIN regions r ON r.id=region_id
        WHERE 1=1 
        AND u.id = {input_id}),
        
        t2 AS (SELECT CASE WHEN (SELECT count(*)
                          FROM t1) = 0 THEN title
                    ELSE (SELECT DISTINCT title
                          FROM t1) END AS city_list
        FROM regions)
        
        SELECT DISTINCT city_list
        FROM t2 
        
        
    '''

result_df = pd.read_sql(query, engine)
print(result_df) 

          city_list
0            Москва
1  Липецкая область
2   Республика Саха


Протестируем полученный код на некоторых значениях:

In [6]:
input_ids = [1, 2, 3, 6]  # Список значений input_id

with engine.connect() as connection:
    for input_id in input_ids:
        query = f'''
            WITH t1 AS (SELECT ur.user_id,
                               r.title AS title
            FROM users_regions ur
            JOIN user u ON u.id=ur.user_id
            JOIN regions r ON r.id=region_id
            WHERE 1=1 
            AND u.id = {input_id}),
            
            t2 AS (SELECT CASE WHEN (SELECT count(*)
                              FROM t1) = 0 THEN title
                        ELSE (SELECT DISTINCT title
                              FROM t1) END AS city_list
            FROM regions)
            
            SELECT DISTINCT city_list
            FROM t2 
        '''

        result_df = pd.read_sql(query, engine)
        print(f"Result for input_id {input_id}:")
        print(result_df)
        print()

Result for input_id 1:
  city_list
0    Москва

Result for input_id 2:
          city_list
0  Липецкая область

Result for input_id 3:
          city_list
0            Москва
1  Липецкая область
2   Республика Саха

Result for input_id 6:
          city_list
0            Москва
1  Липецкая область
2   Республика Саха



## 1.2 Задача про мастеров
По аналогии с задачей 1.1 подготовим данные и напишем SQL-запрос по условию:
>Требуется вывести для каждого мастера клиентов, которые чаще всего были у этого мастера за текущий месяц (если несколько вывести несколько, если ни одного вывести только мастера

In [7]:
#таблица мастеров
masters = {
    'id': [1, 2, 3],
    'last_name': ['Иванов', 'Любовинская', 'Иванович'],
    'first_name': ['Василий', 'Виктория', 'Алексеевна'],
    'middle_name': ['Иванович', 'Дмитрий', 'Олегович'] 
    
}
masters = pd.DataFrame(masters) 
#таблица клиентов
clients = {
    'id': [1, 2, 3],
    'last_name': ['Иванищев', 'Иванищева', 'Пескова'],
    'first_name': ['Петр', 'Мария', 'Ольга'],
    'middle_name': ['Андреевич', 'Андреевна', ''] 
    
}
clients = pd.DataFrame(clients) 
#таблица посещений 
visits = {
    'id': [1, 2, 3, 4, 5],
    'master_id': [1, 1, 2, 1, 1],
    'client_id': [1, 2, 1,2, 3],
    'date': ['25-05-2023', '17-05-2023', '04-05-2022', '26-05-2022', '26-04-2022'] 
}
visits = pd.DataFrame(visits)

visits['date'] = pd.to_datetime(visits['date'], format = "%d-%m-%Y")  

display(masters)
display(clients)
display(visits)

,id,last_name,first_name,middle_name
0,1,Иванов,Василий,Иванович
1,2,Любовинская,Виктория,Дмитрий
2,3,Иванович,Алексеевна,Олегович


,id,last_name,first_name,middle_name
0,1,Иванищев,Петр,Андреевич
1,2,Иванищева,Мария,Андреевна
2,3,Пескова,Ольга,


,id,master_id,client_id,date
0,1,1,1,2023-05-25
1,2,1,2,2023-05-17
2,3,2,1,2022-05-04
3,4,1,2,2022-05-26
4,5,1,3,2022-04-26


Таблицы созданы, зальём их в тестовую базу данных: 

In [8]:
#сохраняем наши таблицы в БД
masters.to_sql('masters', engine, if_exists='replace', index=False)
clients.to_sql('clients', engine, if_exists='replace', index=False)
visits.to_sql('visits', engine, if_exists='replace', index=False) 

5

In [9]:
#для решения используем несколько временных таблиц
#в t1 с данными о визитах фильтруем посещения по текущему месяцу
#в t2 считаем сколько было посещений у мастера по каждому клиенту в текущем месяце,
#в t3 джойним к посещеням расчёты по кол-ву визитов, а также считаем максимальное кол-во посещений оконной функциией
#для случаев если несколько клиентов имели максимальное кол-во
#в t4 джойним данные по клиентам и мастерам и фльтруем, так, что бы остались клиенты с максимальным кол-вом посещений
#в финальному запросе выводим все имена мастеров и клиентов, которые были у него чаще всего
#если клиентов не будет, то рядом с фамилей мастера выводим пустую строку при помощи функции coalesce()


with engine.connect() as connection:
    query = f'''
    WITH t1 AS (
        SELECT *
        FROM visits v 
        WHERE strftime('%m', date(v.date)) = strftime('%m', 'now')
        ),
        t2 AS(
        SELECT master_id,
               client_id,
               COUNT(client_id) AS cnt_visits
        FROM t1 
        GROUP BY 
        1,2
        ),
        
        t3 AS (
        SELECT t1.master_id,
               t1.client_id,
               t2.cnt_visits,
               MAX(t2.cnt_visits) OVER (PARTITION BY t2.master_id) AS max_vists 
        FROM t1
        LEFT JOIN t2 ON t1.master_id=t2.master_id AND t1.client_id = t2.client_id
        ),
        
        t4 AS (
        
        SELECT *,
               m.last_name || ' ' || m.first_name || ' ' || m.middle_name  AS full_name_master,
               c.last_name || ' ' || c.first_name || ' ' || c.middle_name  AS full_name_client 
        FROM t3
        LEFT JOIN masters m ON m.id = t3.master_id 
        LEFT JOIN clients c ON c.id=t3.client_id
        WHERE cnt_visits = max_vists
        )
        
        SELECT DISTINCT
               full_name_master,
               coalesce(full_name_client, '')  AS full_name_client
        FROM t4
    '''

result_df = pd.read_sql(query, engine)
display(result_df)   

,full_name_master,full_name_client
0,Иванов Василий Иванович,Иванищева Мария Андреевна
1,Любовинская Виктория Дмитрий,Иванищев Петр Андреевич


## 2 Визуализация данных
*Входные данные:*
Есть данные о доле домохозяйств, обеспеченных широкополосным доступом (ШПД) к сети Интернет, входящих в состав муниципальных образований РФ, которые в свою очередь входят в регионы РФ в разрезе месяца.

*Требуется:* 
Предложить вариант(ы) (макеты) дашборда для руководителя, содержащего агрегированную информацию о доле домохозяйств, обеспеченных ШПД в рамках страны, а также детальную информацию по регионам и муниципальным образованиям (районам) РФ, с описанием интерактивных сценариев.


**Вариант решения:**

Описание элементов дашборда:

1. Общая информация о доле домохозяйств, обеспеченных ШПД в РФ за текущий месяц. Этот блок содержит информацию о процентном соотношении домохозяйств с ШПД и без ШПД, а также общее количество домохозяйств в РФ.

2. Карта РФ с отображением доли домохозяйств, обеспеченных ШПД, по регионам. Элемент интерактивный, при наведении на регион отображается информация о процентном соотношении домохозяйств с ШПД и без ШПД в этом регионе.

3. Таблица с информацией о доле домохозяйств, обеспеченных ШПД, по муниципальным образованиям (районам) выбранного региона. Элемент интерактивный, при выборе региона на карте в таблице отображается информация о доле домохозяйств с ШПД и без ШПД по районам этого региона.

4. График с динамикой изменения доли домохозяйств, обеспеченных ШПД, в выбранном регионе за текущий месяц по датам. Элемент интерактивный, при выборе другого региона на карте график автоматически обновляется. 

5. Список топ-5 районов выбранного региона с наибольшей долей домохозяйств, обеспеченных ШПД. Элемент интерактивный, при выборе другого региона на карте список автоматически обновляется.

Интерактивные сценарии:

1. Выбор периода. Пользователь может выбрать период, за который будет отображаться информация на дашборде (например, за последние 3 месяца).

2. Выбор региона. Пользователь может выбрать регион на карте, и на основе этого выбора обновятся данные в таблице, графике и списке топ-5 районов.

3. Фильтр по доле домохозяйств с ШПД. Пользователь может задать фильтр по доле домохозяйств с ШПД (например, от 50% и выше), и на основе этого фильтра обновятся данные в таблице, графике и списке топ-5 районов.

**Вариант визуализации:**

![Макет дашборда](https://i.imgur.com/DwqFHsI.png) 

-----

Буду рад получить обратную связь по итоговом тестового задания, спасибо за ваше время!

С уважением, Герасимов К.М. 

Моя электронная почта: gerasimov.km@yandex.ru

Ник в tg: @flackostillrappin 

Телефон: +7(930)-866-66-88

[Ссылка на резюме](https://disk.yandex.ru/i/gRJUdIBzKMiwog)